In [1]:
import numpy as np
from scipy.io import loadmat
SA_raw_data = loadmat('sub_a.mat')
SA_signals = SA_raw_data['responses']
SA_label = SA_raw_data['is_stimulate']
SA_data = []
SA_target = []
for i in range(12):
    for j in range(85):
        if SA_label[i, j] == 1:
            SA_data.append(SA_signals[i, :, :, j].reshape(-1, 64))
            SA_target.append(SA_label[i, j])
            SA_data.append(SA_signals[i, :, :, j].reshape(-1, 64))
            SA_target.append(SA_label[i, j])
            SA_data.append(SA_signals[i, :, :, j].reshape(-1, 64))
            SA_target.append(SA_label[i, j])
            SA_data.append(SA_signals[i, :, :, j].reshape(-1, 64))
            SA_target.append(SA_label[i, j])
        SA_data.append(SA_signals[i, :, :, j].reshape(-1, 64))
        SA_target.append(SA_label[i, j])
SA_data = np.array(SA_data)
SA_target = np.array(SA_target)

C:\Users\wangk\AppData\Roaming\Python\Python36\site-packages\scipy\__init__.py:144: UserWarning: Numpy 1.13.3 or above is required for this version of scipy (detected version 1.13.1)
  UserWarning)


In [2]:
print(SA_target)

[0 1 1 ..., 0 0 0]


In [3]:
SB_raw_data = loadmat('sub_b.mat')
SB_signals = SB_raw_data['responses']
SB_label = SB_raw_data['is_stimulate']
SB_data = []
SB_target = []
for i in range(12):
    for j in range(85):
        if SB_label[i, j] == 1:
            SB_data.append(SB_signals[i, :, :, j].reshape(-1, 64))
            SB_target.append(SB_label[i, j])
            SB_data.append(SB_signals[i, :, :, j].reshape(-1, 64))
            SB_target.append(SB_label[i, j])
            SB_data.append(SB_signals[i, :, :, j].reshape(-1, 64))
            SB_target.append(SB_label[i, j])
            SB_data.append(SB_signals[i, :, :, j].reshape(-1, 64))
            SB_target.append(SB_label[i, j])
        SB_data.append(SB_signals[i, :, :, j].reshape(-1, 64))
        SB_target.append(SB_label[i, j])
SB_data = np.array(SB_data)
SB_target = np.array(SB_target)

In [4]:
def get_num(array):
    num_0=0
    num_1=0
    for i in range(len(array)):
        if array[i]==0:
            num_0+=1
        else:
            num_1+=1
    return num_0,num_1
SA_0,SA_1=get_num(SB_target)
print(SA_0)
print(SA_1)

850
850


In [5]:
print(SA_data.shape)
print(SA_target.shape)
print(SB_data.shape)
print(SB_target.shape)

(1700, 240, 64)
(1700,)
(1700, 240, 64)
(1700,)


In [6]:
all_data=np.concatenate((SA_data,SB_data),axis=0)
all_target=np.concatenate((SA_target,SB_target),axis=0)

In [7]:
import numpy as np
import _pickle as cp

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras import optimizers
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(all_data, all_target, test_size=0.3, random_state=42)
y_train=np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

Using TensorFlow backend.


In [8]:
NB_SENSOR_CHANNELS = 64 #通道数目
NUM_CLASSES = 2       #类别数
BATCH_SIZE = 64
NUM_FILTERS = 64
FILTER_SIZE = 5
NUM_UNITS_LSTM = 128
STRIDE_SIZE=3 #卷积步长

In [9]:
rmp = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

model = Sequential()
model.add(Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_SIZE, strides=STRIDE_SIZE,activation='relu', kernel_initializer='orthogonal',
                 input_shape=(240, NB_SENSOR_CHANNELS), 
                 name='Conv1D_1'))
model.add(Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_SIZE,strides=STRIDE_SIZE, activation='relu', kernel_initializer='orthogonal', 
                 name='Conv1D_2'))
model.add(Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_SIZE,strides=STRIDE_SIZE, activation='relu', kernel_initializer='orthogonal', 
                 name='Conv1D_3'))
#model.add(Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_SIZE, strides=STRIDE_SIZE, activation='relu', kernel_initializer='orthogonal',
#                 name='Conv1D_4'))
model.add(LSTM(NUM_UNITS_LSTM, return_sequences=True, kernel_initializer='orthogonal', 
               name='LSTM_1'))
model.add(LSTM(NUM_UNITS_LSTM, return_sequences=True, kernel_initializer='orthogonal', 
               name='LSTM_2'))
model.add(Flatten(name='Flatten'))
model.add(Dropout(0.5, name='dropout'))
model.add(Dense(NUM_CLASSES, activation='softmax', kernel_initializer='orthogonal', 
                name='Output'))

model.compile(loss='categorical_crossentropy', optimizer=rmp, metrics=['accuracy'])

print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv1D_1 (Conv1D)            (None, 79, 64)            20544     
_________________________________________________________________
Conv1D_2 (Conv1D)            (None, 25, 64)            20544     
_________________________________________________________________
Conv1D_3 (Conv1D)            (None, 7, 64)             20544     
_________________________________________________________________
LSTM_1 (LSTM)                (None, 7, 128)            98816     
_________________________________________________________________
LSTM_2 (LSTM)                (None, 7, 128)            131584    
_________________________________________________________________
Flatten (Flatten)            (None, 896)               0         
________________________________________________

In [10]:
model.fit(X_train, y_train, epochs=100, batch_size=BATCH_SIZE,verbose=1)

Instructions for updating:
Use tf.cast instead.


TypeError: TF_SessionRun_wrapper: expected all values in input dict to be ndarray

In [ ]:
import time
start =time.clock()
test_pred = np.argmax(model.predict(X_test), axis=1)
test_true = np.argmax(y_test, axis=1)
np.unique(test_pred)
import sklearn.metrics as metrics
print("\tTest accauracy:\t{:.4f} ".format(metrics.accuracy_score(test_true, test_pred)))
print(metrics.f1_score(test_true, test_pred)
end = time.clock()
print('Running time: %s Seconds'%(end-start))

In [ ]:
rmp = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

model2 = Sequential()
model2.add(Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_SIZE, strides=STRIDE_SIZE,activation='relu', kernel_initializer='orthogonal',
                 input_shape=(240, NB_SENSOR_CHANNELS), 
                 name='Conv1D_1'))
model2.add(Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_SIZE,strides=STRIDE_SIZE, activation='relu', kernel_initializer='orthogonal', 
                 name='Conv1D_2'))
model2.add(Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_SIZE,strides=STRIDE_SIZE, activation='relu', kernel_initializer='orthogonal', 
                 name='Conv1D_3'))
#model.add(Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_SIZE, strides=STRIDE_SIZE, activation='relu', kernel_initializer='orthogonal',
#                 name='Conv1D_4'))
model2.add(Flatten(name='Flatten1'))
model2.add(Dropout(0.5, name='dropout1'))
model2.add(Dense(128, activation='softmax', kernel_initializer='orthogonal', 
                name='Dense1'))
model2.add(Dropout(0.5, name='dropout2'))
model2.add(Dense(128, activation='softmax', kernel_initializer='orthogonal', 
                name='Dense2'))
model2.add(Dropout(0.5, name='dropout3'))
model2.add(Dense(NUM_CLASSES, activation='softmax', kernel_initializer='orthogonal', 
                name='Output'))

model2.compile(loss='categorical_crossentropy', optimizer=rmp, metrics=['accuracy'])

print(model2.summary())

In [ ]:
model2.fit(X_train, y_train, epochs=100, batch_size=BATCH_SIZE,verbose=1)

In [ ]:
test2_pred = np.argmax(model2.predict(X_test), axis=1)
test_true = np.argmax(y_test, axis=1)
np.unique(test2_pred)
import sklearn.metrics as metrics
print("\tTest accauracy:\t{:.4f} ".format(metrics.accuracy_score(test_true, test2_pred)))
print(metrics.f1_score(test_true, test2_pred)

In [ ]:
print(X_train.shape)
print(X_test.shape)